In [1]:
from src.metagpt import BioformatsReader
import javabridge
import bioformats
import ome_types

javabridge.start_vm(class_path=bioformats.JARS)

image_path = "/home/aaron/Documents/Projects/MetaGPT/in/images/small_images/Rio9 0528.tif"
ome_xml = BioformatsReader.get_omexml_metadata(image_path)
ome_raw = BioformatsReader.get_raw_metadata(image_path)
ome_tree = BioformatsReader.raw_to_tree(ome_raw)
ome_dict = ome_types.to_dict(ome_xml)
print(ome_dict)

javabridge.kill_vm()

16:21:34.132 [Thread-0] DEBUG loci.common.NIOByteBufferProvider - Using mapped byte buffer? false
16:21:34.153 [Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.URLReader
java.lang.ClassNotFoundException: loci.formats.in.URLReader
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:331)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:264)
	at loci.formats.ClassList.parseLine(ClassList.java:196)
	at loci.formats.ClassList.parseFile(ClassList.java:258)
	at loci.formats.ClassList.<init>(ClassList.java:138)
16:21:34.156 [Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.SlideBook6Reader
java.lang.ClassNotFoundException: loci.formats.in.SlideBook6Reader
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lan

In [2]:
# One shot prediction of the structured annotations node
from src.metagpt.predictors.predictor_xml_annotation import PredictorXMLAnnotation

predictor = PredictorXMLAnnotation(str(ome_tree))
prediction = predictor.predict()
print(prediction)

queued
requires_action
RequiredActionFunctionToolCall(id='call_cjvLIHf0wrTeWz161pbrNW5p', function=Function(arguments='{"xml_annotations":{"XResolution":"2728469.0","PhotometricInterpretation":"BlackIsZero","PlanarConfiguration":"Chunky","MetaDataPhotometricInterpretation":"Monochrome","ImageLength":"3072","BitsPerSample":"16","ResolutionUnit":"Inch","Compression":"LZW","SamplesPerPixel":"1","NumberOfChannels":"1","MetaMorph":"no","ImageWidth":"3072","Orientation":"1st row -> top; 1st column -> left","YResolution":"2728469.0"}}', name='StructuredAnnotations'), type='function')
{'xml_annotations': {'XResolution': '2728469.0', 'PhotometricInterpretation': 'BlackIsZero', 'PlanarConfiguration': 'Chunky', 'MetaDataPhotometricInterpretation': 'Monochrome', 'ImageLength': '3072', 'BitsPerSample': '16', 'ResolutionUnit': 'Inch', 'Compression': 'LZW', 'SamplesPerPixel': '1', 'NumberOfChannels': '1', 'MetaMorph': 'no', 'ImageWidth': '3072', 'Orientation': '1st row -> top; 1st column -> left', 'Y

In [5]:
prediction

{'xml_annotations': [{'XResolution': '2728469.0',
   'PhotometricInterpretation': 'BlackIsZero',
   'PlanarConfiguration': 'Chunky',
   'MetaDataPhotometricInterpretation': 'Monochrome',
   'ImageLength': '3072',
   'BitsPerSample': '16',
   'ResolutionUnit': 'Inch',
   'Compression': 'LZW',
   'SamplesPerPixel': '1',
   'NumberOfChannels': '1',
   'MetaMorph': 'no',
   'ImageWidth': '3072',
   'Orientation': '1st row -> top; 1st column -> left',
   'YResolution': '2728469.0'}]}

In [9]:
import ast
from ome_types._autogenerated.ome_2016_06 import XMLAnnotation

#tool = prediction.required_action.submit_tool_outputs.tool_calls[0]
#value = ast.literal_eval(tool.function.arguments)

def dict_to_xml_annotation(value:dict) -> XMLAnnotation:
    fun = lambda k,v : {'qname': '{http://www.openmicroscopy.org/Schemas/OME/2016-06}OriginalMetadata',
                        'text': '',
                        'children': [{'qname': '{http://www.openmicroscopy.org/Schemas/OME/2016-06}Key',
                                      'text': k}, {'qname': '{http://www.openmicroscopy.org/Schemas/OME/2016-06}Value',
                                                   'text': v}]}
    if isinstance(value["xml_annotations"], list):
        value["xml_annotations"] = value["xml_annotations"][0]
    test_value = {"any_elements": [fun(k, v) for k, v in value["xml_annotations"].items()]}
    return XMLAnnotation(value=test_value)

xml = dict_to_xml_annotation(prediction)
import ome_types
ome_start_obj = ome_types.from_xml(ome_xml)
print(ome_start_obj.structured_annotations)
ome_start_obj.structured_annotations.append(xml)
print(ome_start_obj.structured_annotations)
print(ome_types.to_xml(ome_start_obj.structured_annotations))

xml_annotations=[<14 field_type>]
xml_annotations=[<15 field_type>]
<StructuredAnnotations xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
  <XMLAnnotation ID="Annotation:0" Namespace="openmicroscopy.org/OriginalMetadata">
    <Value>
      <OriginalMetadata>
        <Key>PlanarConfiguration</Key>
        <Value>Chunky</Value>
      </OriginalMetadata>
    </Value>
  </XMLAnnotation>
  <XMLAnnotation ID="Annotation:1" Namespace="openmicroscopy.org/OriginalMetadata">
    <Value>
      <OriginalMetadata>
        <Key>XResolution</Key>
        <Value>2728469.0</Value>
      </OriginalMetadata>
    </Value>
  </XMLAnnotation>
  <XMLAnnotation ID="Annotation:2" Namespace="openmicroscopy.org/OriginalMetadata">
    <Value>
      <OriginalMetadata>
        <Key>NumberOfChannels</Key>
        <Value>1</V